In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib notebook
import matplotlib as mpl
mpl.rcParams['figure.figsize'] = 10, 10
from matplotlib.pyplot import imshow
from numpy.linalg import norm
import rawpy
import PIL

#dummyvalues
r_mond=3000. #r_mond in meter

Function to find the radius of the moon in pixel via Sekanten-Verfahren

In [2]:
R= lambda theta: np.array([[np.cos(theta), -np.sin(theta)],
                          [np.sin(theta), np.cos(theta)]])

def mittelsenkrechte(XX,YY):
    '''
    Input: XX, YY: x- und y-koordinaten der zwei Punkte
    Output: mittelpunkt und vektor entlang mittelsenkrechte
    #Output: Funktion die mittelsenkrechte bechreibt als f(t)=vec1+t*vec2
    '''
    R90=R(np.pi*.5)
    koord1=np.array([XX[0],YY[0]])
    koord2=np.array([XX[1], YY[1]])
    mitte=0.5*(koord1+koord2)
    verbindung=koord2-koord1
    mitte_vec=np.dot(R90,verbindung)
    mitte_vec/=norm(mitte_vec)
    return mitte, mitte_vec
    #return lambda t: mitte+t*mitte_vec

def schnittpunkt(m1, v1, m2, v2):
    #findet t&s für den Schnittpunkt der geraden m1+t*v1 und m2+s*v2
    m=m1-m2
    A=np.array([-v1,v2]).T
    return np.linalg.solve(A,m)

def radius_finder(X_werte, Y_werte):
    '''
    Findet Radius des Mondes in Pixeln, via Sekanten
    Input: X_werte, Y_werte: x- und y- Koordinaten der Sekanten-Endpunkte
    Output: radius: Radisu des Mondes in Pixeln
    '''
    radius=0
    nx=len(X_werte)
    if nx!=len(Y_werte):
        print('Fehler')
        return 0
    
    mitte1,vec1=mittelsenkrechte(X_werte[:2], Y_werte[:2])
    mitte2,vec2=mittelsenkrechte(X_werte[round(nx*0.5-0.2):], Y_werte[round(nx*0.5-0.2):])
    t=schnittpunkt(mitte1, vec1, mitte2, vec2)
    kreismittelpunkt=(mitte1+t[0]*vec1+mitte2+t[1]*vec2)*0.5
    
    for i in range(nx):
        radius+=norm(np.array([X_werte[i], Y_werte[i]])-kreismittelpunkt)
    return radius/nx, kreismittelpunkt

Funktion mit der man das Beta auf der Skizzen (Proposal, Whatsapp Chat) finden kann

In [3]:
def betafinder(crater_bild, mitte_bild, unit_vec_laenge, pixeltometer=1):
    '''
        Findet beta strich auf skizze und hoehe in meter (H_mess) des kraters
        crater_bild, mitte_bild sind pixelkoordinaten von Krater und Mondmitte
        unit_vec_laenge ist pixel vektor entlang Nullmeridian, Schatten 'links' vom Vektor,
        muss nicht zwingend normiert sein.
    '''
    unit_vec_laenge=unit_vec_laenge/norm(unit_vec_laenge)
    unit_vec_breite=np.array([unit_vec_laenge[1],-unit_vec_laenge[0]])
    transformmatrix=np.array([unit_vec_laenge,unit_vec_breite]).T
    crater=crater_bild-mitte_bild
    relative_coord=np.linalg.solve(transformmatrix, crater)
    #print(relative_coord)
    hoehe=relative_coord[0]; breite=relative_coord[1] 
    r_strich=np.sqrt(r_mond**2-(hoehe*pixeltometer)**2)
    
    beta_strich=np.arcsin(breite*pixeltometer/r_strich)
    return beta_strich, hoehe*pixeltometer

Given three points (begin crater, begin shadow, end shadow/crater) we calculate the real diameter and height.

In [4]:
def infos_crater(XX,YY, beta, Phi, H_mess, pixeltometer=1):
    coord=np.array([XX,YY])
    s_mess=norm(coord[:,2]-coord[:,1])*pixeltometer
    D_mess=norm(coord[:,2]-coord[:,0])*pixeltometer
    theta=np.arccos(H_mess/r_mond)#klein phi in proposal
    D_real=D_mess/np.sin(theta)
    h=s_mess/np.sin(Phi)*sin(beta)
    return D_real, h

Definition unseres Koordinatensystems.
Berechnet die Koordinaten eines beliebigen Punktes.

In [5]:
'''
    Berechnung des Koordinatensystems:
    Unser Koordinatensystems wird definiert durch die Verbindungslinie der beiden Schattenenden 
    sowie deren Mittelsenkrechte.
'''
XY1=np.array([295.1471251207431,1067.6753550407457])   #unterer Endpunkt Schattenlinie
XY2=np.array([685.4211838861307,97.96379065939959])    #oberer Endpunkt Schattenlinie

origin = .5*(XY1+XY2)
a_y=origin
b_y=(XY2-XY1)
a_x=origin
b_x=np.array([XY1[1]-XY2[1], XY2[0]-XY1[0]])

x_axis = lambda t: a_x + t*b_x    # x-Achse (Verbindungsgerade)
y_axis = lambda t: a_y + t*b_y    # y-Achse (Senkrechte auf Verbindungsgerade)


def coordinates(x,y):
    
    '''
        Input: Pixel-koordinaten des markierten Punktes
        Output: Koordinaten des Punktes in unserem Koordinatensystem
    '''

    p = np.array([x,y])

    t = (b_y[1]*(p[0]-a_x[0]) + b_y[0]*(a_x[1]-p[1]))/(b_x[0]*b_y[1] - b_x[1]*b_y[0])
    t_tilde = (a_x[1] + t*b_x[1] - p[1])/b_y[1]

    x_coord = norm(x_axis(t) - a_x)
    if t<0:
        x_coord=-x_coord
    y_coord = norm(p - x_axis(t))
    if t_tilde>0:
        y_coord = -y_coord

    return x_coord, y_coord

Berechnet Beta-Doppelstrich (Winkel zwischen y-Achse und Schattenlinie). Beta=Beta-Strich+Beta-Doppelstrich

In [6]:
''' 
    Referenzpunkt auf Schattenlinie wurde ausgewählt bei (-149.6596924677921, -0.0797116473548863)
'''
x=abs(-149.6596924677921)    # Abstand Origin-Schattengrenze auf Höhe der x-Achse
r_moon=530.7659439087208     # berechnet mit radius_finder
beta_const = 2*np.arcsin(x/(2*r_moon))
print('rad: ',beta_const)
print('deg: ',360/(2*np.pi)*beta_const)

rad:  0.2829118396366194
deg:  16.20965438546025
